# Countdown Numbers Game

***

## Overview of Countdown Numbers Game

***

In [6]:
#Imports

# Permutations and combinations.
import itertools as it

# Random number generation.
import random

# Operators as functions.
import operator

In [7]:
# Generate Random Game Numbers
def new_numbers_game(no_large=None):
  # Returns six numbers and a target number representing a Countdown numbers game.
  
  # If no_large in None, randomly pick value between 0 and 4 inclusive.
  if no_large is None:
    # Randomly set the value.
    no_large = random.randrange(0, 5)
  
  # Select random large numbers.
  large_rand = random.sample([25, 50, 75, 100], no_large)
  # Select random small numbers.
  small_rand = random.sample(list(range(1, 11)) * 2, 6 - no_large)
  # The playing numbers.
  play_nos = large_rand + small_rand

  # Select a target number.
  target = random.randrange(101, 1000)

  # Return the game.
  return play_nos, target

In [8]:
# Random nubmers game.
new_numbers_game()

([100, 50, 25, 75, 8, 7], 391)

## Complexity of Countdown Numbers Game

***

## Functional Aspect of Code behind Countdown Numbers Game

***

## References

***